# E2: Percent of gdp per country
## - Mimic SUM() OVER() window function in PostgreSQL

In [1]:
import pandas as pd
import numpy as np

In [2]:
summer_games = pd.read_csv("summer_games.csv")
winter_games = pd.read_csv("winter_games.csv")
countries = pd.read_csv("countries.csv")
athletes = pd.read_csv("athletes.csv")
country_stats = pd.read_csv("country_stats.csv")

In [3]:
display(summer_games.head(2))
display(winter_games.head(2))
display(countries.head(2))
display(athletes.head(2))
display(country_stats.head(2))

,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Gymnastics,Gymnastics Men's Individual All-Around,2016-01-01,51,173,NaN,NaN,NaN
1,Gymnastics,Gymnastics Men's Floor Exercise,2016-01-01,51,173,NaN,NaN,NaN


,sport,event,year,athlete_id,country_id,bronze,silver,gold
0,Alpine Skiing,Alpine Skiing Women's Slalom,2014-01-01,126,89,NaN,NaN,NaN
1,Alpine Skiing,Alpine Skiing Women's Super G,2014-01-01,463,102,NaN,NaN,NaN


,id,country,region
0,1,AFG - Afghanistan,ASIA (EX. NEAR EAST)
1,2,ALB - Albania,EASTERN EUROPE


,id,name,gender,age,height,weight
0,51,Nstor Abad Sanjun,M,23,167.0,64.0
1,55,Antonio Abadia Beci,M,26,170.0,65.0


,year,country_id,gdp,population,nobel_prize_winners
0,2000-01-01,1,NaN,20093756.0,NaN
1,2000-01-01,2,3.632044e+09,3089027.0,NaN


In [14]:
gdp_region_country = \
countries.merge(country_stats, left_on = "id", right_on="country_id")\
         .loc[lambda df: ~df['gdp'].isna(), ['country','region', 'gdp']]\
         .groupby(['country', 'region'])['gdp'].sum().reset_index()

In [15]:
gdp_region_country.head()

,country,region,gdp
0,ANT - Antigua and Barbuda,LATIN AMER. & CARIB,1.910801e+10
1,AUS - Australia,OCEANIA,1.621277e+13
2,BEL - Belgium,WESTERN EUROPE,7.179396e+12
3,BRN - Bahrain,NEAR EAST,3.711941e+11
4,BIZ - Belize,LATIN AMER. & CARIB,2.238559e+10


In [50]:
'''assign(region_gdp = lambda df: df.groupby('region')['gdp'].transform('sum')) # create a new column region_gdp 
which is the sum gdp value grouped by each region but returns a column has same axis length as it's base df, 
which is important in this case to perform a later calculation '''

final = \
gdp_region_country.assign(total_gdp = lambda df: df['gdp'].sum())\
                  .assign(perc_global_gdp = lambda df: df['gdp'].div(df['total_gdp']))\
                  .assign(region_gdp = lambda df: df.groupby('region')['gdp'].transform('sum'))\
                  .assign(perc_region_gdp = lambda df: df.apply(lambda row: row['gdp']/row['region_gdp'], axis=1))\
                  .sort_values('gdp', ascending=False)

In [51]:
final

,country,region,gdp,total_gdp,perc_global_gdp,region_gdp,perc_region_gdp
188,USA - United States,NORTHERN AMERICA,2.437361e+14,9.714136e+14,2.509087e-01,2.667923e+14,0.913580
37,CHN - China,ASIA (EX. NEAR EAST),9.051442e+13,9.714136e+14,9.317805e-02,2.551111e+14,0.354804
92,JPN - Japan,ASIA (EX. NEAR EAST),8.404848e+13,9.714136e+14,8.652183e-02,2.551111e+14,0.329458
70,GER - Germany,WESTERN EUROPE,5.299421e+13,9.714136e+14,5.455371e-02,2.561369e+14,0.206898
66,GBR - Great Britain,WESTERN EUROPE,4.205792e+13,9.714136e+14,4.329559e-02,2.561369e+14,0.164201
...,...,...,...,...,...,...,...
33,CAY - Cayman Islands,LATIN AMER. & CARIB,3.207033e+09,9.714136e+14,3.301408e-06,7.002880e+13,0.000046
165,STP - Sao Tome and Principe,SUB-SAHARAN AFRICA,3.135309e+09,9.714136e+14,3.227573e-06,1.769066e+13,0.000177
118,MHL - Marshall Islands,OCEANIA,2.618646e+09,9.714136e+14,2.695707e-06,1.882197e+13,0.000139
96,KIR - Kiribati,OCEANIA,2.251355e+09,9.714136e+14,2.317607e-06,1.882197e+13,0.000120
